In [ ]:
# from datasets import load_dataset
# df = load_dataset("xmanii/Persian-Math-SFT", split="train").to_pandas()
# df.head()

In [ ]:
# مرحله ۱: نصب پکیج‌ها (حدود ۲-۴ دقیقه طول می‌کشه)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes xformers datasets



  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hzg_97qb/unsloth_75b983c52b3b41ac9c80b0c8a2038aed
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hzg_97qb/unsloth_75b983c52b3b41ac9c80b0c8a2038aed
  Resolved https://github.com/unslothai/unsloth.git to commit d8459b194c81a502962f16118509cdfa205483af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 138.7 MB/s eta 0:00:00

In [ ]:
# مرحله ۲: ایمپورت‌ها
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:


# مرحله ۳: تنظیمات مهم
max_seq_length = 2048      # برای ریاضی معمولاً کافیه
dtype = None               # None = auto (bf16 اگه پشتیبانی بشه)
load_in_4bit = True        # مهم برای صرفه‌جویی حافظه

# مرحله ۴: انتخاب مدل (اینجا ۷ بی هست - می‌تونی عوض کنی)
model_name = "Qwen/Qwen2.5-7B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name              = model_name,
    max_seq_length          = max_seq_length,
    dtype                   = dtype,
    load_in_4bit            = load_in_4bit,
    # trust_remote_code     = True   # معمولاً برای Qwen2.5 لازم نیست
)

# مرحله ۵: اضافه کردن LoRA (فقط این بخش‌ها رو یاد می‌گیره)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,                    # ۱۶ تا ۶۴ خوبه - بزرگتر = بهتر ولی حافظه بیشتر
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,          # بهینه‌ترین حالت
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # خیلی مهم برای صرفه‌جویی حافظه
    random_state = 3407,
    max_seq_length = max_seq_length,
)

==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:


# مرحله ۶: لود کردن دیتاست فارسی ریاضی
dataset = load_dataset("xmanii/Persian-Math-SFT", split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# قبل از تعریف trainer این دو بخش را اضافه کن

from unsloth.chat_templates import get_chat_template

# ۱. چت تمپلیت مناسب Qwen2.5 را به tokenizer اضافه کن
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",          # مخصوص Qwen2.5
    # mapping = None   # معمولاً لازم نیست چون Qwen استاندارد است
)


# ۲. تابع فرمت‌بندی (اینجا مهم‌ترین قسمت است)
def formatting_prompts_func(examples):
    # examples یک بچ از دیتاست است
    texts = []
    for messages in examples["messages"]:
        # apply_chat_template خودش بهترین فرمت رو برای Qwen می‌سازه
        text = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = False   # موقع آموزش False باشد
        )
        texts.append(text)

    return { "text" : texts }


# ۳. دیتاست رو map کن → ستون "text" ساخته می‌شود
dataset = dataset.map(
    formatting_prompts_func,
    batched = True,
    # num_proc = 4,       # اگر دیتاست بزرگ بود سرعت می‌دهد
)


# حالا trainer رو این شکلی تعریف کن (قسمت dataset_text_field عوض شد)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",           # ← حالا "text" هست نه "messages"
    max_seq_length = max_seq_length,
    packing = True,                        # همچنان می‌تونی نگه داری
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 20,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Qwen2.5-7B-PersianMath-unsloth",
        report_to = "none",
    ),
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:

# مرحله ۸: شروع آموزش
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 80,740,352 of 7,696,356,864 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,0.661200
40,0.543600
60,0.508700
80,0.485700
100,0.469100
120,0.454800
140,0.445200
160,0.413300
180,0.431900
200,0.420400


TrainOutput(global_step=250, training_loss=0.46905184745788575, metrics={'train_runtime': 822.0165, 'train_samples_per_second': 2.433, 'train_steps_per_second': 0.304, 'total_flos': 1.167115599051817e+17, 'train_loss': 0.46905184745788575, 'epoch': 2.0})

In [ ]:

# مرحله ۹: ذخیره مدل (LoRA adapter)
model.save_pretrained("Qwen2.5-7B-PersianMath-LoRA")
tokenizer.save_pretrained("Qwen2.5-7B-PersianMath-LoRA")

('Qwen2.5-7B-PersianMath-LoRA/tokenizer_config.json',
 'Qwen2.5-7B-PersianMath-LoRA/special_tokens_map.json',
 'Qwen2.5-7B-PersianMath-LoRA/chat_template.jinja',
 'Qwen2.5-7B-PersianMath-LoRA/vocab.json',
 'Qwen2.5-7B-PersianMath-LoRA/merges.txt',
 'Qwen2.5-7B-PersianMath-LoRA/added_tokens.json',
 'Qwen2.5-7B-PersianMath-LoRA/tokenizer.json')

In [ ]:
from google.colab import files

# If you saved LoRA adapter (most common with Unsloth)
# files.download('/content/Qwen2.5-7B-PersianMath-LoRA/adapter_model.bin')
# files.download('/content/Qwen2.5-7B-PersianMath-LoRA/adapter_config.json')
# files.download('/content/Qwen2.5-7B-PersianMath-LoRA/README.md')  # optional

# Or download the whole folder as zip (more convenient)
!zip -r persian-math-lora.zip /content/Qwen2.5-7B-PersianMath-LoRA
files.download('persian-math-lora.zip')

  adding: content/Qwen2.5-7B-PersianMath-LoRA/ (stored 0%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/adapter_model.safetensors (deflated 8%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/special_tokens_map.json (deflated 69%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/added_tokens.json (deflated 67%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/adapter_config.json (deflated 57%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/tokenizer_config.json (deflated 89%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/merges.txt (deflated 57%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/chat_template.jinja (deflated 71%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/README.md (deflated 65%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/vocab.json (deflated 61%)
  adding: content/Qwen2.5-7B-PersianMath-LoRA/tokenizer.json (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained_merged("Qwen2.5-7B-PersianMath-merged", tokenizer, save_method = "merged_16bit")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:41<00:00, 10.30s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:52<00:00, 13.25s/it]


Unsloth: Merge process complete. Saved to `/content/Qwen2.5-7B-PersianMath-merged`


In [ ]:
!zip -r persian-math-merged.zip /content/Qwen2.5-7B-PersianMath-merged
files.download('persian-math-merged.zip')

updating: content/Qwen2.5-7B-PersianMath-merged/ (stored 0%)
updating: content/Qwen2.5-7B-PersianMath-merged/special_tokens_map.json (deflated 69%)
updating: content/Qwen2.5-7B-PersianMath-merged/added_tokens.json (deflated 67%)
updating: content/Qwen2.5-7B-PersianMath-merged/tokenizer_config.json (deflated 83%)
updating: content/Qwen2.5-7B-PersianMath-merged/model-00004-of-00004.safetensors (deflated 22%)
updating: content/Qwen2.5-7B-PersianMath-merged/config.json (deflated 74%)
updating: content/Qwen2.5-7B-PersianMath-merged/model-00003-of-00004.safetensors (deflated 21%)
updating: content/Qwen2.5-7B-PersianMath-merged/model.safetensors.index.json (deflated 95%)
updating: content/Qwen2.5-7B-PersianMath-merged/merges.txt (deflated 57%)
updating: content/Qwen2.5-7B-PersianMath-merged/chat_template.jinja (deflated 71%)
updating: content/Qwen2.5-7B-PersianMath-merged/.cache/ (stored 0%)
updating: content/Qwen2.5-7B-PersianMath-merged/.cache/huggingface/ (stored 0%)
updating: content/Qwen

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r persian-math-unsloth.zip /content/Qwen2.5-7B-PersianMath-unsloth
files.download('persian-math-unsloth.zip')

In [ ]:

model.push_to_hub("arefehRajabian/Qwen2.5-7B-PersianMath-LoRA", token=True)
tokenizer.push_to_hub("arefehRajabian/Qwen2.5-7B-PersianMath-LoRA", token=True)

print("✅ Model and tokenizer pushed successfully!")